In [1]:
import torch
import torch.nn as nn
import os
import warnings
import json
import torch.nn.functional as F
import torch.optim as optim
import configparser
from tqdm import tqdm
from train_tools import EarlyStopping,get_logger
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
from transformers import BertTokenizer,Adafactor,MT5ForConditionalGeneration
from torch.utils.data.dataloader import DataLoader
from data_prepare import data_prepare,process_answer
from dataset_and_model import JecT5CombDataset,JecT5Model

In [48]:
line = "1Cats are smarter [sep] than dogs 1[sep] thancsds 1[sep]"
# searchObj = re.search('^.* than', line)
searchObj = re.search('^.*(\[sep\]) than', line)


In [49]:
searchObj.group(0)

'1Cats are smarter [sep] than dogs 1[sep] than'

In [80]:
ss = 'adafasw12314egrdf5236qew'
num = re.search('\d+',ss)

In [81]:
num

<re.Match object; span=(7, 12), match='12314'>

In [133]:
line.split('[sep]')

['1Cats are smarter ', ' than dogs 1', ' ']

In [50]:
try:
    a==sf
except:
    print(1)

1


In [2]:
tokenizer = BertTokenizer.from_pretrained("fnlp/bart-large-chinese")
model = BartForConditionalGeneration.from_pretrained("fnlp/bart-large-chinese")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [27]:
# a = '[unused1]手动阀[unused2]啊萨芬[unused3]'
a = '!'
tk = tokenizer(a,return_tensors='pt')

In [28]:
tk.pop('token_type_ids')

tensor([[0, 0, 0]])

In [29]:
tk

{'input_ids': tensor([[101, 106, 102]]), 'attention_mask': tensor([[1, 1, 1]])}

In [16]:
# tk['input_ids'].unsqueeze_(0)
# tk['attention_mask'].unsqueeze_(0)

In [6]:
p = model.generate(**tk)

In [7]:
p

tensor([[  102,   101,   138,   163, 11316,  8303,  8148,   140,  2797,  1220,
          7322,   138,   163, 10287,  8303,  8144,   140,  1557,  5855,   102],
        [  102,   101,   138,   163, 11316,  8303,  8148,   140,  2797,  1220,
          7322,   138,   163, 10287,  8303,  8144,   140,  1557,  5855,   102]])

In [8]:
ou = tokenizer.decode(p[0], skip_special_tokens=True)

In [9]:
o = re.search('\[ unused1 \].*?', ou)

In [10]:
o.group()

'[ unused1 ]'

In [9]:
ou

'[ unused1 ] 手 动 阀 [ uradeed2 ] 啊 萨'

In [17]:
ps = '[ unused1 ] 手 动 阀 [ unused2 ] 啊 萨'
o = re.search('\[ unused1 \].*?\[ unused2 \]', ps)

In [18]:
o.group()

'[ unused1 ] 手 动 阀 [ unused2 ]'

In [10]:
ou.replace(' ','')

'[unused1]手动阀[uradeed2]啊萨'

In [8]:
target = [[1,1,0,1],[1,0,0,0],[0,1,1,1],[0,1,0,0]]
idx = [i for i in range(len(target)) if target[i].count(1)==1]


In [9]:
idx

[1, 3]

In [20]:
small = 0
start_evaluate = 0 if small else 10


In [21]:
start_evaluate

10

In [2]:
config = configparser.ConfigParser()
config.read("config.ini", encoding="utf-8")
section_name = 't5'

batch_size = config.getint(section_name,'batch_size')
# epochs = config.getint(section_name,'epochs')
small_sample = 1
if small_sample:
    epochs = 1
else:
    epochs = config.getint(section_name,'epochs')
device = torch.device('cuda:'+config.get(section_name,'gpu') if torch.cuda.is_available else 'cpu')
lr = config.getfloat(section_name,'batch_size')
accum_step = config.getint(section_name,'accum_step')
patience = config.getint(section_name,'patience')
model_name = config.get(section_name,'model_name')
model_savepath = config.get(section_name,'model_savepath')
predict_savepath = config.get(section_name,'predict_savepath')
log_path = config.get(section_name,'log_path')
evaluate_path = config.get(section_name,'evaluate_path')

train_datapath = config.get('DEFAULT','train_datapath')
valid_datapath = config.get('DEFAULT','valid_datapath')
test_datapath = config.get('DEFAULT','test_datapath')


# In[3]:


train_data,valid_data,test_data = data_prepare(train_datapath,test_datapath)


# In[4]:


tokenizer = BertTokenizer.from_pretrained(model_name)


# In[5]:

if small_sample:
    train_data = JecT5CombDataset(train_data[:40],tokenizer)
    valid_data = JecT5CombDataset(valid_data[:40],tokenizer)
else:
    train_data = JecT5CombDataset(train_data,tokenizer)
    valid_data = JecT5CombDataset(valid_data,tokenizer)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 17.01it/s]


In [3]:
tokenizer('The <extra_id_0> walks in <extra_id_1> park', return_tensors='pt').input_ids


tensor([[  101,   100,   133,  9577,  9808,   142,  8231,   142,   121,   135,
           165, 11346,  8118,  8217,   133,  9577,  9808,   142,  8231,   142,
           122,   135,  8865,   102]])

In [5]:
tokenizer('extra0', return_tensors='pt').input_ids

tensor([[  101, 21128,   102]])